In [2]:
import pandas as pd
import numpy as np

print("="*60)
print("FINALIZING FEATURE SET FOR MODEL TRAINING")
print("="*60)

# Load your training data
df = pd.read_csv('nba_train_data.csv')
print(f"\nOriginal dataset: {df.shape[0]} games, {df.shape[1]} features")

# ============================================================
# 1. ✅ Create NET_RATING_DIFF
# ============================================================
print("\n1️⃣ Creating NET_RATING_DIFF...")
df['NET_RATING_DIFF'] = df['HOME_NET_RATING_PRIOR'] - df['AWAY_NET_RATING_PRIOR']
print(f"   ✓ NET_RATING_DIFF created")
print(f"     Range: {df['NET_RATING_DIFF'].min():.2f} to {df['NET_RATING_DIFF'].max():.2f}")
print(f"     Mean: {df['NET_RATING_DIFF'].mean():.2f}")
print(f"     Correlation with HOME_WIN: {df['NET_RATING_DIFF'].corr(df['HOME_WIN']):.4f}")

# ============================================================
# 2. ✅ Create SEASON_PROGRESS features
# ============================================================
print("\n2️⃣ Creating SEASON_PROGRESS features...")
REGULAR_SEASON_GAMES = 82

df['HOME_SEASON_PROGRESS'] = (df['HOME_GP_PRIOR'] / REGULAR_SEASON_GAMES).clip(upper=1.0)
df['AWAY_SEASON_PROGRESS'] = (df['AWAY_GP_PRIOR'] / REGULAR_SEASON_GAMES).clip(upper=1.0)

print(f"   ✓ HOME_SEASON_PROGRESS created")
print(f"     Range: {df['HOME_SEASON_PROGRESS'].min():.3f} to {df['HOME_SEASON_PROGRESS'].max():.3f}")
print(f"   ✓ AWAY_SEASON_PROGRESS created")
print(f"     Range: {df['AWAY_SEASON_PROGRESS'].min():.3f} to {df['AWAY_SEASON_PROGRESS'].max():.3f}")

# ============================================================
# 3. ✅ Create Four Factors Differentials
# ============================================================
print("\n3️⃣ Creating Four Factors Differentials...")

# Effective Field Goal % (40% weight in Four Factors)
df['EFG_PCT_DIFF'] = df['HOME_EFG_PCT_FF_PRIOR'] - df['AWAY_EFG_PCT_FF_PRIOR']
print(f"   ✓ EFG_PCT_DIFF created (Shooting Efficiency - 40% weight)")
print(f"     Correlation with HOME_WIN: {df['EFG_PCT_DIFF'].corr(df['HOME_WIN']):.4f}")

# Turnover % (25% weight) - Lower is better, so we invert
df['TOV_PCT_DIFF'] = df['AWAY_TM_TOV_PCT_FF_PRIOR'] - df['HOME_TM_TOV_PCT_FF_PRIOR']
print(f"   ✓ TOV_PCT_DIFF created (Turnovers - 25% weight, inverted)")
print(f"     Correlation with HOME_WIN: {df['TOV_PCT_DIFF'].corr(df['HOME_WIN']):.4f}")

# Offensive Rebound % (20% weight)
df['OREB_PCT_DIFF'] = df['HOME_OREB_PCT_FF_PRIOR'] - df['AWAY_OREB_PCT_FF_PRIOR']
print(f"   ✓ OREB_PCT_DIFF created (Rebounding - 20% weight)")
print(f"     Correlation with HOME_WIN: {df['OREB_PCT_DIFF'].corr(df['HOME_WIN']):.4f}")

# Free Throw Rate (15% weight)
df['FTA_RATE_DIFF'] = df['HOME_FTA_RATE_PRIOR'] - df['AWAY_FTA_RATE_PRIOR']
print(f"   ✓ FTA_RATE_DIFF created (Free Throws - 15% weight)")
print(f"     Correlation with HOME_WIN: {df['FTA_RATE_DIFF'].corr(df['HOME_WIN']):.4f}")

# ============================================================
# 4. Create REST_ADVANTAGE Buckets
# ============================================================
print("\n4️⃣ Creating REST_ADVANTAGE Buckets...")

# First calculate REST_ADVANTAGE if it doesn't exist
if 'REST_ADVANTAGE' not in df.columns:
    df['REST_ADVANTAGE'] = df['HOME_DAYS_REST'] - df['AWAY_DAYS_REST']
    print(f"   ✓ REST_ADVANTAGE calculated first")

# Create bucketed version
df['REST_BUCKET'] = pd.cut(df['REST_ADVANTAGE'], 
                            bins=[-np.inf, -1.5, -0.5, 0.5, 1.5, np.inf],
                            labels=['Away_Big_Adv', 'Away_Slight_Adv', 'Equal', 
                                    'Home_Slight_Adv', 'Home_Big_Adv'])

print(f"   ✓ REST_BUCKET created (5 categories)")
print(f"\n   Distribution:")
print(df['REST_BUCKET'].value_counts().sort_index())
print(f"\n   Win rates by bucket:")
print(df.groupby('REST_BUCKET')['HOME_WIN'].agg(['mean', 'count']).round(3))

# Create dummy variables for REST_BUCKET (for models that need numeric input)
rest_dummies = pd.get_dummies(df['REST_BUCKET'], prefix='REST', drop_first=True)
df = pd.concat([df, rest_dummies], axis=1)
print(f"   ✓ REST_BUCKET dummy variables created: {list(rest_dummies.columns)}")

# ============================================================
# 5. Create Interaction Features
# ============================================================
print("\n5️⃣ Creating Interaction Features...")

# NET_RATING_DIFF × SEASON_PROGRESS
# Theory: Net rating becomes more reliable later in season
df['NET_RATING_x_SEASON'] = df['NET_RATING_DIFF'] * df['HOME_SEASON_PROGRESS']
print(f"   ✓ NET_RATING_x_SEASON created")
print(f"     Correlation with HOME_WIN: {df['NET_RATING_x_SEASON'].corr(df['HOME_WIN']):.4f}")

# B2B × NET_RATING interaction
# Theory: Good teams may handle B2B better
df['HOME_B2B_x_NET_RATING'] = df['HOME_B2B'] * df['NET_RATING_DIFF']
print(f"   ✓ HOME_B2B_x_NET_RATING created")
print(f"     Correlation with HOME_WIN: {df['HOME_B2B_x_NET_RATING'].corr(df['HOME_WIN']):.4f}")

# ============================================================
# 6. Additional Useful Differentials
# ============================================================
print("\n6️⃣ Creating Additional Differentials...")

# Win percentage differential (if not exists)
if 'WIN_PCT_DIFF' not in df.columns:
    df['WIN_PCT_DIFF'] = df['HOME_W_PCT_PRIOR'] - df['AWAY_W_PCT_PRIOR']
    print(f"   ✓ WIN_PCT_DIFF created")

# Defensive Rating differential
df['DEF_RATING_DIFF'] = df['AWAY_DEF_RATING_PRIOR'] - df['HOME_DEF_RATING_PRIOR']  # Inverted (lower is better)
print(f"   ✓ DEF_RATING_DIFF created (inverted - lower is better)")
print(f"     Correlation with HOME_WIN: {df['DEF_RATING_DIFF'].corr(df['HOME_WIN']):.4f}")

# Offensive Rating differential
df['OFF_RATING_DIFF'] = df['HOME_OFF_RATING_PRIOR'] - df['AWAY_OFF_RATING_PRIOR']
print(f"   ✓ OFF_RATING_DIFF created")
print(f"     Correlation with HOME_WIN: {df['OFF_RATING_DIFF'].corr(df['HOME_WIN']):.4f}")

# Pace differential (game tempo)
df['PACE_DIFF'] = df['HOME_PACE_PRIOR'] - df['AWAY_PACE_PRIOR']
print(f"   ✓ PACE_DIFF created")
print(f"     Correlation with HOME_WIN: {df['PACE_DIFF'].corr(df['HOME_WIN']):.4f}")

# ============================================================
# 7. Summary of New Features
# ============================================================
print("\n" + "="*60)
print("FEATURE ENGINEERING COMPLETE")
print("="*60)

new_features = [
    'NET_RATING_DIFF', 'HOME_SEASON_PROGRESS', 'AWAY_SEASON_PROGRESS',
    'EFG_PCT_DIFF', 'TOV_PCT_DIFF', 'OREB_PCT_DIFF', 'FTA_RATE_DIFF',
    'REST_BUCKET', 'NET_RATING_x_SEASON', 'HOME_B2B_x_NET_RATING',
    'WIN_PCT_DIFF', 'DEF_RATING_DIFF', 'OFF_RATING_DIFF', 'PACE_DIFF'
] + list(rest_dummies.columns)

print(f"\n✅ Created {len(new_features)} new features")
print(f"\nNew dataset shape: {df.shape[0]} games, {df.shape[1]} features")
print(f"Feature count increase: +{df.shape[1] - 177}")

# ============================================================
# 8. Define Final Feature Set for Modeling
# ============================================================
print("\n" + "-"*60)
print("FEATURE LIST BY TIER:")
print("-"*60)

# Count features by tier to get correct indices
tier_structure = [
    ("TIER 1: Critical Predictors (NET_RATING)", [
        'NET_RATING_DIFF',
        'HOME_NET_RATING_PRIOR',
        'AWAY_NET_RATING_PRIOR',
    ]),
    ("TIER 2: Rest/Fatigue (15.9% swing)", [
        'HOME_B2B',
        'AWAY_B2B',
        'REST_ADVANTAGE',
    ]),
    ("TIER 3: Four Factors Differentials (Dean Oliver)", [
        'EFG_PCT_DIFF',
        'TOV_PCT_DIFF',
        'OREB_PCT_DIFF',
        'FTA_RATE_DIFF',
    ]),
    ("TIER 4: Four Factors Individual Teams", [
        'HOME_EFG_PCT_FF_PRIOR',
        'AWAY_EFG_PCT_FF_PRIOR',
        'HOME_TM_TOV_PCT_FF_PRIOR',
        'AWAY_TM_TOV_PCT_FF_PRIOR',
        'HOME_OREB_PCT_FF_PRIOR',
        'AWAY_OREB_PCT_FF_PRIOR',
        'HOME_FTA_RATE_PRIOR',
        'AWAY_FTA_RATE_PRIOR',
    ]),
    ("TIER 5: Win Percentage", [
        'WIN_PCT_DIFF',
        'HOME_W_PCT_PRIOR',
        'AWAY_W_PCT_PRIOR',
    ]),
    ("TIER 6: Season Context", [
        'HOME_SEASON_PROGRESS',
        'AWAY_SEASON_PROGRESS',
    ]),
    ("TIER 7: Offense/Defense Split", [
        'OFF_RATING_DIFF',
        'DEF_RATING_DIFF',
        'HOME_OFF_RATING_PRIOR',
        'AWAY_OFF_RATING_PRIOR',
        'HOME_DEF_RATING_PRIOR',
        'AWAY_DEF_RATING_PRIOR',
    ]),
    ("TIER 8: Pace & Style", [
        'PACE_DIFF',
        'HOME_PACE_PRIOR',
        'AWAY_PACE_PRIOR',
    ]),
    ("TIER 9: Rest Details", [
        'HOME_DAYS_REST',
        'AWAY_DAYS_REST',
    ]),
    ("TIER 10: Interaction Terms", [
        'NET_RATING_x_SEASON',
        'HOME_B2B_x_NET_RATING',
    ]),
    ("TIER 11: REST_BUCKET Dummies", list(rest_dummies.columns)),
]

# Rebuild FINAL_FEATURES in correct order
FINAL_FEATURES = []
feature_counter = 1

for tier_name, tier_features in tier_structure:
    print(f"\n{tier_name}")
    for feature in tier_features:
        print(f"  {feature_counter:2d}. {feature}")
        FINAL_FEATURES.append(feature)
        feature_counter += 1

print(f"\n{'='*60}")
print(f"Total Features: {len(FINAL_FEATURES)}")
print(f"{'='*60}")

# ============================================================
# 9. Feature Correlation Analysis
# ============================================================
print("\n" + "="*60)
print("TOP 15 FEATURES BY CORRELATION WITH HOME_WIN")
print("="*60)

correlations = df[FINAL_FEATURES + ['HOME_WIN']].corr()['HOME_WIN'].drop('HOME_WIN').abs().sort_values(ascending=False)
print("\n", correlations.head(15).to_string())

# ============================================================
# 10. Save Enhanced Dataset
# ============================================================
print("\n" + "="*60)
print("SAVING ENHANCED DATASET")
print("="*60)

# Save full enhanced dataset
output_filename = 'nba_train_data_enhanced.csv'
df.to_csv(output_filename, index=False)
print(f"\n✓ Full enhanced dataset saved: {output_filename}")
print(f"  Shape: {df.shape}")

# Save feature list for reference
feature_list_filename = 'model_features.txt'
with open(feature_list_filename, 'w') as f:
    f.write("="*60 + "\n")
    f.write("FINAL FEATURE SET FOR NBA PREDICTION MODEL\n")
    f.write("="*60 + "\n\n")
    f.write(f"Total Features: {len(FINAL_FEATURES)}\n")
    f.write(f"Target Variable: HOME_WIN\n\n")
    f.write("Features (in order):\n")
    f.write("-"*60 + "\n")
    for i, feature in enumerate(FINAL_FEATURES, 1):
        f.write(f"{i:2d}. {feature}\n")

print(f"✓ Feature list saved: {feature_list_filename}")

# Create a summary statistics file
summary_filename = 'feature_summary_stats.csv'
feature_stats = df[FINAL_FEATURES].describe().T
feature_stats['correlation_with_HOME_WIN'] = correlations
feature_stats.to_csv(summary_filename)
print(f"✓ Feature summary stats saved: {summary_filename}")

print("\n" + "="*60)
print("✅ FEATURE ENGINEERING COMPLETE!")
print("="*60)
print("\n📊 Summary:")
print(f"   • Original features: 177")
print(f"   • New features created: {len(new_features)}")
print(f"   • Total features now: {df.shape[1]}")
print(f"   • Features selected for modeling: {len(FINAL_FEATURES)}")
print(f"\n📁 Files Created:")
print(f"   1. {output_filename}")
print(f"   2. {feature_list_filename}")
print(f"   3. {summary_filename}")
print("\n🚀 Ready for model training!")

FINALIZING FEATURE SET FOR MODEL TRAINING

Original dataset: 5085 games, 177 features

1️⃣ Creating NET_RATING_DIFF...
   ✓ NET_RATING_DIFF created
     Range: -43.20 to 42.70
     Mean: -0.10
     Correlation with HOME_WIN: 0.3082

2️⃣ Creating SEASON_PROGRESS features...
   ✓ HOME_SEASON_PROGRESS created
     Range: 0.012 to 0.988
   ✓ AWAY_SEASON_PROGRESS created
     Range: 0.012 to 0.988

3️⃣ Creating Four Factors Differentials...
   ✓ EFG_PCT_DIFF created (Shooting Efficiency - 40% weight)
     Correlation with HOME_WIN: 0.2026
   ✓ TOV_PCT_DIFF created (Turnovers - 25% weight, inverted)
     Correlation with HOME_WIN: 0.1531
   ✓ OREB_PCT_DIFF created (Rebounding - 20% weight)
     Correlation with HOME_WIN: -0.0438
   ✓ FTA_RATE_DIFF created (Free Throws - 15% weight)
     Correlation with HOME_WIN: 0.0167

4️⃣ Creating REST_ADVANTAGE Buckets...
   ✓ REST_BUCKET created (5 categories)

   Distribution:
REST_BUCKET
Away_Big_Adv        213
Away_Slight_Adv     821
Equal           

C:\Users\userPC\AppData\Local\Temp\ipykernel_4160\794318681.py:81: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df.groupby('REST_BUCKET')['HOME_WIN'].agg(['mean', 'count']).round(3))



✓ Full enhanced dataset saved: nba_train_data_enhanced.csv
  Shape: (5085, 195)
✓ Feature list saved: model_features.txt
✓ Feature summary stats saved: feature_summary_stats.csv

✅ FEATURE ENGINEERING COMPLETE!

📊 Summary:
   • Original features: 177
   • New features created: 18
   • Total features now: 195
   • Features selected for modeling: 40

📁 Files Created:
   1. nba_train_data_enhanced.csv
   2. model_features.txt
   3. feature_summary_stats.csv

🚀 Ready for model training!


In [3]:
print("="*60)
print("DATA LEAKAGE CHECK: Verifying _PRIOR Features Usage")
print("="*60)

# Check what we actually used in our differential calculations
print("\n🔍 CHECKING EACH DIFFERENTIAL FOR DATA LEAKAGE:\n")

# 1. NET_RATING_DIFF
print("1. NET_RATING_DIFF:")
print(f"   Formula used: HOME_NET_RATING_PRIOR - AWAY_NET_RATING_PRIOR")
print(f"   ✅ Uses _PRIOR features - NO LEAKAGE")

# 2. Four Factors Differentials
print("\n2. EFG_PCT_DIFF:")
print(f"   Formula used: HOME_EFG_PCT_FF_PRIOR - AWAY_EFG_PCT_FF_PRIOR")
print(f"   ✅ Uses _PRIOR features - NO LEAKAGE")

print("\n3. TOV_PCT_DIFF:")
print(f"   Formula used: AWAY_TM_TOV_PCT_FF_PRIOR - HOME_TM_TOV_PCT_FF_PRIOR")
print(f"   ✅ Uses _PRIOR features - NO LEAKAGE")

print("\n4. OREB_PCT_DIFF:")
print(f"   Formula used: HOME_OREB_PCT_FF_PRIOR - AWAY_OREB_PCT_FF_PRIOR")
print(f"   ✅ Uses _PRIOR features - NO LEAKAGE")

print("\n5. FTA_RATE_DIFF:")
print(f"   Formula used: HOME_FTA_RATE_PRIOR - AWAY_FTA_RATE_PRIOR")
print(f"   ✅ Uses _PRIOR features - NO LEAKAGE")

# 3. Other Differentials
print("\n6. WIN_PCT_DIFF:")
print(f"   Formula used: HOME_W_PCT_PRIOR - AWAY_W_PCT_PRIOR")
print(f"   ✅ Uses _PRIOR features - NO LEAKAGE")

print("\n7. DEF_RATING_DIFF:")
print(f"   Formula used: AWAY_DEF_RATING_PRIOR - HOME_DEF_RATING_PRIOR")
print(f"   ✅ Uses _PRIOR features - NO LEAKAGE")

print("\n8. OFF_RATING_DIFF:")
print(f"   Formula used: HOME_OFF_RATING_PRIOR - AWAY_OFF_RATING_PRIOR")
print(f"   ✅ Uses _PRIOR features - NO LEAKAGE")

print("\n9. PACE_DIFF:")
print(f"   Formula used: HOME_PACE_PRIOR - AWAY_PACE_PRIOR")
print(f"   ✅ Uses _PRIOR features - NO LEAKAGE")

# 4. Check that we're NOT using any non-PRIOR features
print("\n" + "="*60)
print("VERIFYING: Checking for accidental non-PRIOR usage")
print("="*60)

# List all columns that DON'T have _PRIOR and aren't metadata/target
non_prior_features = [col for col in df.columns if '_PRIOR' not in col 
                      and col not in ['GAME_ID', 'GAME_DATE', 'SEASON', 
                                     'HOME_TEAM_ID', 'HOME_TEAM_ABBREVIATION', 'HOME_TEAM_NAME',
                                     'AWAY_TEAM_ID', 'AWAY_TEAM_ABBREVIATION', 'AWAY_TEAM_NAME',
                                     'HOME_WIN', 'HOME_WL', 'AWAY_WL',
                                     'HOME_B2B', 'AWAY_B2B', 'REST_ADVANTAGE', 'HOME_DAYS_REST', 'AWAY_DAYS_REST',
                                     'REST_BUCKET', 'NET_RATING_DIFF', 'EFG_PCT_DIFF', 'TOV_PCT_DIFF',
                                     'OREB_PCT_DIFF', 'FTA_RATE_DIFF', 'WIN_PCT_DIFF', 'DEF_RATING_DIFF',
                                     'OFF_RATING_DIFF', 'PACE_DIFF', 'HOME_SEASON_PROGRESS', 'AWAY_SEASON_PROGRESS',
                                     'NET_RATING_x_SEASON', 'HOME_B2B_x_NET_RATING', 'SEASON_STAGE']
                      and not col.startswith('REST_')]

# Check if any of these are in FINAL_FEATURES
dangerous_features = [f for f in FINAL_FEATURES if f in non_prior_features]

if dangerous_features:
    print("\n⚠️  WARNING: Found potential data leakage features:")
    for feat in dangerous_features:
        print(f"   ❌ {feat}")
else:
    print("\n✅ NO DATA LEAKAGE DETECTED!")
    print("   All features use _PRIOR values or legitimate pre-game info")

# 5. Detailed verification with sample data
print("\n" + "="*60)
print("SAMPLE VERIFICATION: First Game Example")
print("="*60)

sample = df.iloc[0]
print(f"\nGame: {sample['HOME_TEAM_ABBREVIATION']} vs {sample['AWAY_TEAM_ABBREVIATION']}")
print(f"Date: {sample['GAME_DATE']}")
print(f"Actual Result: HOME_WIN = {sample['HOME_WIN']}")

print(f"\n📊 Features used for prediction (all pre-game info):")
print(f"   HOME_NET_RATING_PRIOR: {sample['HOME_NET_RATING_PRIOR']:.2f}")
print(f"   AWAY_NET_RATING_PRIOR: {sample['AWAY_NET_RATING_PRIOR']:.2f}")
print(f"   → NET_RATING_DIFF: {sample['NET_RATING_DIFF']:.2f}")

print(f"\n   HOME_EFG_PCT_FF_PRIOR: {sample['HOME_EFG_PCT_FF_PRIOR']:.3f}")
print(f"   AWAY_EFG_PCT_FF_PRIOR: {sample['AWAY_EFG_PCT_FF_PRIOR']:.3f}")
print(f"   → EFG_PCT_DIFF: {sample['EFG_PCT_DIFF']:.3f}")

print(f"\n   HOME_B2B: {sample['HOME_B2B']}")
print(f"   AWAY_B2B: {sample['AWAY_B2B']}")

print(f"\n   HOME_GP_PRIOR: {sample['HOME_GP_PRIOR']:.0f} games")
print(f"   → HOME_SEASON_PROGRESS: {sample['HOME_SEASON_PROGRESS']:.3f}")

# 6. Final Summary
print("\n" + "="*60)
print("DATA LEAKAGE CHECK: FINAL SUMMARY")
print("="*60)
print("\n✅ ALL CLEAR - No data leakage detected!")
print("\nAll differentials correctly use _PRIOR features:")
print("   • NET_RATING_DIFF ← uses _PRIOR")
print("   • Four Factors Diffs ← use _PRIOR") 
print("   • WIN_PCT_DIFF ← uses _PRIOR")
print("   • OFF/DEF_RATING_DIFF ← use _PRIOR")
print("   • PACE_DIFF ← uses _PRIOR")
print("\nLegitimate pre-game info (not leakage):")
print("   • HOME/AWAY_B2B ← known before game")
print("   • HOME/AWAY_DAYS_REST ← known before game")
print("   • SEASON_PROGRESS ← calculated from GP_PRIOR")
print("\n🎯 Your feature engineering is TEMPORALLY SOUND!")

DATA LEAKAGE CHECK: Verifying _PRIOR Features Usage

🔍 CHECKING EACH DIFFERENTIAL FOR DATA LEAKAGE:

1. NET_RATING_DIFF:
   Formula used: HOME_NET_RATING_PRIOR - AWAY_NET_RATING_PRIOR
   ✅ Uses _PRIOR features - NO LEAKAGE

2. EFG_PCT_DIFF:
   Formula used: HOME_EFG_PCT_FF_PRIOR - AWAY_EFG_PCT_FF_PRIOR
   ✅ Uses _PRIOR features - NO LEAKAGE

3. TOV_PCT_DIFF:
   Formula used: AWAY_TM_TOV_PCT_FF_PRIOR - HOME_TM_TOV_PCT_FF_PRIOR
   ✅ Uses _PRIOR features - NO LEAKAGE

4. OREB_PCT_DIFF:
   Formula used: HOME_OREB_PCT_FF_PRIOR - AWAY_OREB_PCT_FF_PRIOR
   ✅ Uses _PRIOR features - NO LEAKAGE

5. FTA_RATE_DIFF:
   Formula used: HOME_FTA_RATE_PRIOR - AWAY_FTA_RATE_PRIOR
   ✅ Uses _PRIOR features - NO LEAKAGE

6. WIN_PCT_DIFF:
   Formula used: HOME_W_PCT_PRIOR - AWAY_W_PCT_PRIOR
   ✅ Uses _PRIOR features - NO LEAKAGE

7. DEF_RATING_DIFF:
   Formula used: AWAY_DEF_RATING_PRIOR - HOME_DEF_RATING_PRIOR
   ✅ Uses _PRIOR features - NO LEAKAGE

8. OFF_RATING_DIFF:
   Formula used: HOME_OFF_RATING_PRIO